In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
# import pywhatkit
from datetime import datetime

pygame 2.3.0 (SDL 2.24.2, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'C:\Users\Kashmira\Implemented code\model.h5')

In [5]:
mixer.init()
sound = mixer.Sound(r'C:\Users\Kashmira\Implemented code\alarm.wav')
video_capture = cv2.VideoCapture(0)
eyes_closed_duration = 0;
while True:
    ret , frame = video_capture.read()
    height , width = frame.shape[0:2]
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2 , minNeighbors = 3 )
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor = 1.1 , minNeighbors = 2 )
    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, pt1=(x,y), pt2=(x+w,y+h) , color = (255,0,0) , thickness = 3)

    for (ex,ey,ew,eh) in eyes:
        #cv2.rectangle(frame, pt1=(ex,ey), pt2=(ex+ew,ey+eh) , color = (255,0,0) , thickness = 3)
        #preprocessing step
        #------------------------------------
        eye = frame[ey:ey+eh,ex:ex+eh]
        eye = cv2.resize(eye,(80,80)) 
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye,axis=0)
        #------------------------------------
        #model prediction
        prediction = model.predict(eye)
        
        #if eyes are closed
        if prediction[0][0] > 0.30 :
            cv2.putText(
                        frame,
                        'Eyes Closed',
                        (10,height-20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1,
                        color = (255,255,255),
                        thickness = 1,
                        lineType = cv2.LINE_AA
            )
            cv2.putText(
                        frame,
                        'Eyes Closed for '+str(eyes_closed_duration)+' ms',
                        (10,height-20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1,
                        color = (255,255,255),
                        thickness = 1,
                        lineType = cv2.LINE_AA
            )
            eyes_closed_duration = eyes_closed_duration+1
            
            if( eyes_closed_duration > 5 ):
                try:
                    sound.play()
                    message = 'Driver is drowsed!!'+str(datetime.now())
#                     pywhatkit.sendwhatmsg('+91 7722037711',message,10,47)
                except:
                    pass
                    
        #eyes are open
        elif prediction[0][1] > 0.90 :
            cv2.putText(
                        frame,
                        'Eyes Open',
                        (10,height-20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1,
                        color = (0,255,0),
                        thickness = 1,
                        lineType = cv2.LINE_AA
            )
            eyes_closed_duration = eyes_closed_duration-1
            if(eyes_closed_duration < 0 ):
                eyes_closed_duration = 0
        #print(prediction)
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()
    
    

1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 42ms/step
